In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [2]:
df = DataFrame(XLSX.readtable("usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [3]:
colnames(data_source)

11-element Vector{Symbol}:
 :Yeld_10Y
 :Price_10Y
 :Stock_Price
 :Dividend
 :E
 :CAPE
 :T_Bill
 :CPI
 :Dividend_Yeld
 :Total_Stock_2
 :Total_Stock

In [4]:
freq = 4
#:Price_10Y
returns = percentchange(data_source[:Total_Stock], :log)
#returns = merge(returns, log1p.(data_source[:T_Bill])./freq)
cpi = percentchange(data_source[:CPI], :log)
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
term = data_source[:Yeld_10Y] ./freq .- data_source[:T_Bill] ./freq
dy = log.(data_source[:Dividend_Yeld])
data = merge(excess_returns, real_t_bill, term, cpi)

291×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1952-03-01 to 2024-07-01
┌────────────┬────────────────────┬─────────────┬─────────────────┬─────────────
│            │ Total_Stock_T_Bill │ T_Bill_CPI  │ Yeld_10Y_T_Bill │ CPI        ⋯
├────────────┼────────────────────┼─────────────┼─────────────────┼─────────────
│ 1952-03-01 │        -0.00772801 │  0.00772445 │       0.0022875 │ -0.0037807 ⋯
│ 1952-06-01 │          0.0651996 │ -0.00708528 │       0.0017875 │   0.011299 ⋯
│ 1952-09-01 │         -0.0227685 │  0.00423886 │       0.0024375 │         0. ⋯
│ 1952-12-01 │            0.08432 │   0.0089235 │       0.0011425 │ -0.0037523 ⋯
│ 1953-03-01 │         -0.0487217 │  0.00497517 │       0.0018125 │         0. ⋯
│ 1953-06-01 │        -0.00778441 │ -0.00227055 │       0.0021925 │  0.0074906 ⋯
│ 1953-09-01 │        -0.00277599 │ -0.00299956 │        0.002115 │  0.0074349 ⋯
│ 1953-12-01 │          0.0705234 │  0.00767892 │       0.0021725 │ -0.0037105 ⋯
│     ⋮      │         ⋮         

In [5]:
CSV.write("usa_data_transformed.csv", data)

"usa_data_transformed.csv"

In [72]:
start = Date(1998,01,01)
end_d = Date(2019, 12, 31)
s_data = to(from(data, start),end_d)

returns_summarystats(to(from(returns, start),end_d), freq)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1454,0.7736,0.074,0.252,-1.0147,-0.9472,-0.0626,0.1509,1.8816,0.5092,0.1428


In [73]:
returns_summarystats(s_data, 1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.0731,0.1825,0.0138,0.0616,-0.2565,-0.9474,-0.0186,0.1606,1.7048,0.1889,0.0305
T_Bill_CPI,0.0084,0.0255,-0.0007,0.0045,-0.0208,0.3107,-0.0052,0.0571,0.5427,-0.0815,-0.001
Yeld_10Y_T_Bill,0.0028,0.0095,0.0042,0.0063,-0.0016,-0.0991,0.002,0.8849,-0.9889,1.4761,0.0046
CPI,0.0079,0.0236,0.0053,0.0098,-0.0254,-0.6506,0.0018,-0.077,2.3017,0.671,0.0049


In [74]:
returns_summarystats(s_data, freq)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.1462,0.7299,0.0552,0.2465,-1.026,-0.9474,-0.0746,0.1606,1.7048,0.3778,0.1218
T_Bill_CPI,0.0168,0.1019,-0.0027,0.0182,-0.0832,0.3107,-0.0209,0.0571,0.5427,-0.163,-0.0038
Yeld_10Y_T_Bill,0.0057,0.038,0.0168,0.0253,-0.0065,-0.0991,0.0078,0.8849,-0.9889,2.9523,0.0183
CPI,0.0158,0.0946,0.0213,0.0394,-0.1016,-0.6506,0.0071,-0.077,2.3017,1.342,0.0195


In [75]:
vmodel = VARModel(s_data)
model_summary(vmodel)

,const,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI,std
Total_Stock_T_Bill,0.0887,0.1337,-7.1367,-9.7153,-7.8291,0.0684
T_Bill_CPI,-0.0036,-0.0355,0.5888,-0.0585,0.7452,0.0073
Yeld_10Y_T_Bill,0.001,-0.0006,-0.0616,0.8271,-0.0492,0.0013
CPI,0.0034,0.0407,0.3961,0.087,0.2363,0.0074


,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI
Total_Stock_T_Bill,1.0,-0.07,0.14,0.09
T_Bill_CPI,-0.07,1.0,-0.17,-0.99
Yeld_10Y_T_Bill,0.14,-0.17,1.0,0.1
CPI,0.09,-0.99,0.1,1.0


In [76]:
bmodel = NormalWishartBVAR(s_data)
sample_posterior!(bmodel,10000, 2000)
posterior_summary(bmodel)

"Total_Stock_T_Bill coefficients"

Quantiles
          parameters       2.5%      25.0%     50.0%     75.0%     97.5% 
              Symbol    Float64    Float64   Float64   Float64   Float64 

               const     0.0344     0.0699    0.0882    0.1068    0.1418
  Total_Stock_T_Bill    -0.0746     0.0649    0.1366    0.2069    0.3411
          T_Bill_CPI   -11.9785    -8.8409   -7.1204   -5.4913   -2.3334
     Yeld_10Y_T_Bill   -17.3717   -12.3738   -9.7247   -7.0340   -1.9133
                 CPI   -12.6206    -9.4541   -7.7917   -6.1768   -2.9714


"T_Bill_CPI coefficients"

Quantiles
          parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
              Symbol   Float64   Float64   Float64   Float64   Float64 

               const   -0.0095   -0.0056   -0.0036   -0.0016    0.0021
  Total_Stock_T_Bill   -0.0583   -0.0428   -0.0352   -0.0277   -0.0132
          T_Bill_CPI    0.0768    0.4087    0.5847    0.7647    1.1278
     Yeld_10Y_T_Bill   -0.8948   -0.3503   -0.0603    0.2247    0.7800
                 CPI    0.2430    0.5675    0.7393    0.9196    1.2790


"Yeld_10Y_T_Bill coefficients"

Quantiles
          parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
              Symbol   Float64   Float64   Float64   Float64   Float64 

               const   -0.0000    0.0006    0.0009    0.0013    0.0020
  Total_Stock_T_Bill   -0.0045   -0.0019   -0.0006    0.0008    0.0033
          T_Bill_CPI   -0.1528   -0.0925   -0.0618   -0.0305    0.0281
     Yeld_10Y_T_Bill    0.6799    0.7780    0.8284    0.8774    0.9726
                 CPI   -0.1394   -0.0800   -0.0493   -0.0184    0.0395


"CPI coefficients"

Quantiles
          parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
              Symbol   Float64   Float64   Float64   Float64   Float64 

               const   -0.0024    0.0014    0.0034    0.0054    0.0094
  Total_Stock_T_Bill    0.0182    0.0329    0.0405    0.0480    0.0639
          T_Bill_CPI   -0.1466    0.2164    0.4010    0.5811    0.9190
     Yeld_10Y_T_Bill   -0.7705   -0.2014    0.0855    0.3843    0.9443
                 CPI   -0.3073    0.0591    0.2432    0.4165    0.7550


"correlation matrix"

Quantiles
                             parameters      2.5%     25.0%     50.0%     75.0 ⋯
                                 Symbol   Float64   Float64   Float64   Float6 ⋯

  Total_Stock_T_Bill_Total_Stock_T_Bill    0.0603    0.0663    0.0699    0.073 ⋯
          Total_Stock_T_Bill_T_Bill_CPI   -0.2751   -0.1410   -0.0663    0.007 ⋯
     Total_Stock_T_Bill_Yeld_10Y_T_Bill   -0.0761    0.0672    0.1406    0.213 ⋯
                 Total_Stock_T_Bill_CPI   -0.1263    0.0160    0.0903    0.166 ⋯
          T_Bill_CPI_Total_Stock_T_Bill   -0.2751   -0.1410   -0.0663    0.007 ⋯
                  T_Bill_CPI_T_Bill_CPI    0.0065    0.0071    0.0075    0.007 ⋯
             T_Bill_CPI_Yeld_10Y_T_Bill   -0.3715   -0.2391   -0.1665   -0.092 ⋯
                         T_Bill_CPI_CPI   -0.9950   -0.9934   -0.9923   -0.991 ⋯
     Yeld_10Y_T_Bill_Total_Stock_T_Bill   -0.0761    0.0672    0.1406    0.213 ⋯
             Yeld_10Y_T_Bill_T_Bill_CPI   -0.3715   -0.2391   -0.1665   -0.092 ⋯
        Yeld_10Y_

In [77]:
assets_names = colnames(s_data)
scenarios = simulate(vmodel, freq * 15)

for n in 1:length(assets_names)
   print_percentiles(scenarios[n, :, 2:end], [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], freq, string(assets_names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.1018,-0.2125,-0.2385,-0.2519,-0.2517,-0.2556,-0.2562,-0.2658,-0.2421,-0.2532,-0.2462,-0.2538,-0.254,-0.242,-0.2464
0.05,-0.046,-0.1439,-0.1788,-0.1901,-0.1853,-0.1863,-0.1838,-0.194,-0.178,-0.187,-0.1809,-0.1876,-0.1894,-0.1788,-0.1811
0.25,0.0952,0.0077,-0.018,-0.029,-0.0309,-0.0343,-0.0281,-0.029,-0.0249,-0.0278,-0.0241,-0.025,-0.0223,-0.023,-0.0258
0.5,0.1926,0.1149,0.0906,0.0809,0.0743,0.0769,0.0795,0.0822,0.0833,0.0847,0.0862,0.0861,0.0869,0.0894,0.082
0.75,0.2872,0.2219,0.1977,0.1898,0.187,0.1892,0.1903,0.1924,0.1944,0.1938,0.1978,0.1987,0.1996,0.1993,0.1923
0.95,0.4238,0.3827,0.3452,0.3449,0.3431,0.352,0.3476,0.355,0.3586,0.3523,0.3599,0.3551,0.3593,0.3542,0.3492
0.98,0.4865,0.4434,0.4098,0.4124,0.4108,0.4173,0.4166,0.4243,0.4209,0.4223,0.4196,0.4188,0.423,0.4251,0.4181


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.038,-0.0365,-0.0381,-0.0388,-0.0399,-0.0407,-0.0425,-0.0436,-0.0426,-0.0431,-0.0433,-0.0449,-0.0436,-0.0444,-0.0441
0.05,-0.0331,-0.0302,-0.0318,-0.0324,-0.0335,-0.0339,-0.0354,-0.0361,-0.0361,-0.0366,-0.037,-0.0373,-0.037,-0.0376,-0.0372
0.25,-0.0198,-0.0159,-0.0164,-0.017,-0.0175,-0.0179,-0.019,-0.0192,-0.0196,-0.02,-0.0204,-0.0204,-0.0208,-0.0207,-0.0201
0.5,-0.0105,-0.0062,-0.0057,-0.0061,-0.0062,-0.0066,-0.0075,-0.0079,-0.0077,-0.0084,-0.0086,-0.0084,-0.0091,-0.0088,-0.0084
0.75,-0.0012,0.004,0.0049,0.0047,0.0047,0.0045,0.0039,0.0037,0.0033,0.0036,0.0032,0.0033,0.0026,0.0029,0.0031
0.95,0.0121,0.0184,0.0202,0.0207,0.0211,0.0206,0.0202,0.02,0.0201,0.0202,0.0196,0.0202,0.0197,0.0198,0.0194
0.98,0.0177,0.0246,0.0267,0.028,0.0273,0.0271,0.0271,0.0261,0.0273,0.0271,0.0264,0.0274,0.0267,0.0271,0.0266


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0036,-0.004,-0.003,-0.0028,-0.0024,-0.0027,-0.0025,-0.0017,-0.002,-0.0021,-0.0022,-0.0019,-0.0019,-0.0013,-0.0009
0.05,-0.0014,-0.0003,0.0007,0.0011,0.0014,0.0015,0.0016,0.0024,0.0023,0.0023,0.002,0.0021,0.0024,0.0027,0.0026
0.25,0.0046,0.0077,0.0091,0.0103,0.0107,0.0113,0.0115,0.0118,0.012,0.012,0.0121,0.0122,0.0122,0.0123,0.0123
0.5,0.0087,0.0131,0.0153,0.0166,0.0174,0.0179,0.018,0.0185,0.0187,0.0186,0.0188,0.019,0.019,0.0192,0.0192
0.75,0.0126,0.0187,0.0215,0.0228,0.024,0.0245,0.0247,0.0252,0.0254,0.0254,0.0255,0.0257,0.026,0.026,0.0259
0.95,0.0182,0.0268,0.0302,0.0322,0.0332,0.0337,0.0342,0.0349,0.0349,0.0349,0.0349,0.0355,0.0353,0.0357,0.0357
0.98,0.0205,0.0303,0.0337,0.036,0.0372,0.0378,0.0384,0.0392,0.0393,0.0394,0.039,0.039,0.0395,0.0398,0.0397


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0027,-0.0076,-0.009,-0.0095,-0.0105,-0.0108,-0.01,-0.0103,-0.0116,-0.012,-0.0106,-0.0114,-0.0106,-0.0097,-0.0107
0.05,0.0034,-0.0018,-0.003,-0.0034,-0.004,-0.0045,-0.0037,-0.0045,-0.0054,-0.0048,-0.0047,-0.005,-0.0044,-0.0044,-0.0044
0.25,0.0174,0.0126,0.0114,0.011,0.0107,0.0102,0.0106,0.01,0.0102,0.0098,0.0104,0.0096,0.0102,0.01,0.0098
0.5,0.0272,0.0227,0.0217,0.0213,0.0209,0.0205,0.0206,0.0204,0.0206,0.0203,0.0205,0.02,0.0204,0.0204,0.0198
0.75,0.037,0.0327,0.0321,0.0316,0.031,0.0304,0.0309,0.0309,0.0304,0.0307,0.0307,0.0305,0.0306,0.0306,0.03
0.95,0.0512,0.047,0.0466,0.0458,0.0452,0.0455,0.0454,0.0459,0.0448,0.0456,0.0448,0.0456,0.0458,0.0451,0.0445
0.98,0.0566,0.0528,0.0526,0.0522,0.0514,0.0513,0.0516,0.0522,0.0514,0.0514,0.0509,0.0522,0.0512,0.0511,0.0505


In [99]:
y_data = annualise(scenarios[1, :, 21:25],4)
display("mean")
display(mean(y_data))
display("std")
display(std(y_data))
display("skewness")
display(skewness(y_data))
display("kurtosis")
display(kurtosis(y_data)) 

"mean"

0.07774126165057227

"std"

0.1626815400201295

"skewness"

0.0004931763606276683

"kurtosis"

-0.058496132734868755

In [78]:
scenarios_b = simulate(bmodel,freq * 15, 10)

for n in 1:4
    print_percentiles(scenarios_b[n, :, 2:end], [0.05, 0.25, 0.5, 0.75, 0.95], freq, string(assets_names[n]))
 end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.2491,-0.7485,-0.8735,-0.9334,-0.961,-0.9749,-0.9787,-0.9914,-1.0,-1.0069,-1.0125,-1.0234,-1.0189,-1.0324,-1.0302
0.25,0.018,-0.1988,-0.2607,-0.2836,-0.2932,-0.2984,-0.3062,-0.3048,-0.3037,-0.3055,-0.3084,-0.3143,-0.3127,-0.3138,-0.3092
0.5,0.1755,0.1092,0.09,0.0812,0.079,0.0806,0.081,0.0818,0.0836,0.0847,0.0805,0.0818,0.0829,0.0875,0.0846
0.75,0.3517,0.4274,0.448,0.4535,0.4569,0.464,0.4705,0.4733,0.48,0.4807,0.477,0.4805,0.4837,0.49,0.4891
0.95,0.6704,0.9965,1.0805,1.1149,1.1398,1.1518,1.1758,1.1812,1.1799,1.1878,1.2059,1.209,1.2138,1.2298,1.2348


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0596,-0.0958,-0.1125,-0.123,-0.1307,-0.1362,-0.1425,-0.1454,-0.1501,-0.1534,-0.1563,-0.1588,-0.1621,-0.1645,-0.1672
0.25,-0.0285,-0.039,-0.0453,-0.0492,-0.052,-0.0544,-0.0563,-0.0573,-0.0586,-0.0593,-0.0601,-0.0606,-0.0615,-0.0621,-0.0626
0.5,-0.0097,-0.0054,-0.0051,-0.0055,-0.0061,-0.0066,-0.007,-0.0073,-0.0077,-0.008,-0.0081,-0.0078,-0.008,-0.0084,-0.0087
0.75,0.0083,0.0275,0.0343,0.0381,0.0398,0.0411,0.042,0.0426,0.0426,0.0426,0.0435,0.0442,0.044,0.0442,0.0441
0.95,0.0369,0.0821,0.0993,0.1095,0.117,0.1213,0.1256,0.1293,0.1317,0.135,0.1371,0.1393,0.1411,0.1423,0.1438


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0296,-0.0439,-0.0504,-0.0559,-0.0601,-0.0633,-0.0664,-0.0691,-0.0704,-0.072,-0.0739,-0.0753,-0.0762,-0.0778,-0.0791
0.25,-0.0064,-0.0088,-0.0098,-0.0108,-0.0115,-0.0121,-0.0126,-0.0129,-0.0132,-0.0131,-0.013,-0.0132,-0.0137,-0.0137,-0.0136
0.5,0.0086,0.0131,0.0151,0.0159,0.0169,0.0172,0.0176,0.0179,0.0183,0.0188,0.0188,0.0186,0.0189,0.0189,0.0191
0.75,0.0235,0.0345,0.0398,0.0429,0.0451,0.0467,0.0478,0.0487,0.0496,0.0502,0.0507,0.051,0.0515,0.0517,0.0521
0.95,0.0459,0.0677,0.0786,0.0863,0.0922,0.0966,0.1007,0.1041,0.1068,0.1089,0.1116,0.1141,0.1156,0.1165,0.1178


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0013,-0.0176,-0.0242,-0.0279,-0.0309,-0.0328,-0.0349,-0.0367,-0.0381,-0.0395,-0.0404,-0.0422,-0.0435,-0.0441,-0.0453
0.25,0.0169,0.0085,0.006,0.0046,0.0038,0.0032,0.0029,0.0025,0.0021,0.0017,0.0015,0.0013,0.0012,0.001,0.0008
0.5,0.0274,0.0227,0.0218,0.0214,0.0211,0.0208,0.0208,0.0208,0.0206,0.0204,0.0205,0.0203,0.0203,0.0204,0.0204
0.75,0.0377,0.0369,0.0377,0.0379,0.0382,0.0384,0.0386,0.0388,0.0389,0.0388,0.039,0.0389,0.039,0.0392,0.0392
0.95,0.0544,0.0632,0.0675,0.0702,0.0719,0.0733,0.0751,0.0764,0.0774,0.0785,0.0794,0.0801,0.0811,0.0817,0.0825


In [79]:
# annualised compounded returns for 1, 5, 10, 15 years
for n in 1:3
    cum_ret = cumsum(scenarios[n,:,2:end]; dims=2)
    print_percentiles(expm1.(cum_ret[:,[1 * freq, freq * 5, freq * 10, freq * 15]] ./ [1 5 10 15]),[.02, .1, .25, .5, .75, .9, .98], 1, string(assets_names[n]))    
end

,1,2,3,4
0.02,-0.0968,-0.0134,0.0063,0.0184
0.1,0.01,0.0329,0.0407,0.047
0.25,0.0998,0.0715,0.0689,0.0708
0.5,0.2124,0.1169,0.1014,0.0976
0.75,0.3327,0.1645,0.1345,0.1249
0.9,0.4488,0.2086,0.1646,0.1505
0.98,0.6266,0.2653,0.2034,0.1832


,1,2,3,4
0.02,-0.0373,-0.0221,-0.0215,-0.0215
0.1,-0.0277,-0.0165,-0.0164,-0.0164
0.25,-0.0196,-0.012,-0.0121,-0.0124
0.5,-0.0105,-0.0069,-0.0073,-0.0078
0.75,-0.0012,-0.0018,-0.0024,-0.0032
0.9,0.0073,0.0029,0.0021,0.0011
0.98,0.0178,0.009,0.0078,0.0067


,1,2,3,4
0.02,-0.0036,0.0012,0.0043,0.0062
0.1,0.0011,0.0061,0.0088,0.0103
0.25,0.0046,0.01,0.0125,0.0136
0.5,0.0088,0.0142,0.0164,0.0174
0.75,0.0127,0.0187,0.0204,0.0211
0.9,0.0162,0.0227,0.024,0.0243
0.98,0.0207,0.0277,0.029,0.0289


In [80]:
collect(.05:.1:.95)

10-element Vector{Float64}:
 0.05
 0.15
 0.25
 0.35
 0.45
 0.55
 0.65
 0.75
 0.85
 0.95

In [81]:
for n in 1:3
    print_percentiles(scenarios[n, :, 2:end],[.05, .15, .25, .35, .45, .55, .65, 0.75, .85, 0.95], freq*5, string(assets_names[n]))
end


,1,2,3
0.05,0.0521,-0.1882,-0.1623
0.15,0.2398,0.0329,0.0571
0.25,0.3454,0.1675,0.1873
0.35,0.4335,0.2735,0.2916
0.45,0.5168,0.3652,0.389
0.55,0.593,0.4561,0.4765
0.65,0.6737,0.5514,0.5743
0.75,0.7616,0.6547,0.6794
0.85,0.8712,0.7806,0.8129
0.95,1.0511,1.008,1.0278


,1,2,3
0.05,-0.097,-0.1229,-0.1296
0.15,-0.074,-0.0916,-0.099
0.25,-0.0604,-0.0734,-0.0806
0.35,-0.0494,-0.0584,-0.0653
0.45,-0.0398,-0.0448,-0.0508
0.55,-0.0301,-0.0315,-0.0371
0.65,-0.0204,-0.0186,-0.0234
0.75,-0.009,-0.0038,-0.0081
0.85,0.0052,0.0151,0.0112
0.95,0.0289,0.0455,0.0462


,1,2,3
0.05,0.0191,0.0278,0.0288
0.15,0.0383,0.0511,0.0526
0.25,0.0498,0.0661,0.0678
0.35,0.0588,0.0765,0.0797
0.45,0.0667,0.0869,0.0906
0.55,0.0746,0.097,0.1006
0.65,0.0833,0.1072,0.1111
0.75,0.0925,0.1182,0.1223
0.85,0.1046,0.1322,0.1364
0.95,0.1235,0.1551,0.1596


In [82]:
 cov_m = cov(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI])) 
 mu = mean(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI]), dims=1) 

KeyError: KeyError: key :Price_10Y_T_Bill not found

In [83]:
using Distributions
scenarios_n = zeros(3,100_000,freq*15)

for t in 1:freq*15
    scenarios_n[:,:,t] = rand(MvNormal(vec(mu), cov_m), 100_000)
end



UndefVarError: UndefVarError: `mu` not defined

In [84]:
for n in 1:3
    print_percentiles(scenarios_n[n, :, 1:end],[0.02, 0.25, 0.5, 0.75, 0.98], 12, string(assets_names[n]))
end

,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


In [85]:
returns_summarystats(data_source[:Yeld_10Y],1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Yeld_10Y,0.0292,0.1584,0.0548,0.0736,0.0066,0.9377,0.0333,0.9774,0.6945,1.8772,0.0468


In [86]:
yelds10y = dropdims(sum(scenarios[[3,5,6],:,:], dims=1), dims=1)
print_percentiles(annualise(yelds10y,4), [0.02, 0.25, 0.5, 0.75, 0.98])

BoundsError: BoundsError: attempt to access 4×10000×61 Array{Float64, 3} at index [[3, 5, 6], 1:10000, 1:61]

In [87]:
include("generate_bond_temp.jl")

using .Bonds

bond_returns = Bonds.scenarios_bond_returns(yelds10y, 10, freq)
print_percentiles(bond_returns, [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `yelds10y` not defined

In [88]:
#cpi_ann = annualise(scenarios[6,:,2:end],4)
t_bill = scenarios[3,:,2:end]
excess_bond_returns = bond_returns .- t_bill
print_percentiles(expm1.(excess_bond_returns), [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `bond_returns` not defined

In [89]:
scenario = 3
plot(yelds10y[scenario,:], label="yeld")
plot!(twinx(), bond_returns[scenario,:], label="bond returns", color=:orange)

UndefVarError: UndefVarError: `yelds10y` not defined

In [90]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

5-element Vector{Float64}:
 -0.1828283017775568
 -0.02720428872508454
  0.08059087067324498
  0.18870909840162942
  0.34374424936690545

In [91]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

5-element Vector{Float64}:
 -0.18198598822338113
 -0.027820972919280575
  0.08039647515260423
  0.1882973693421442
  0.34371764880707123

In [92]:
simt = rand(LocationScale(.04, .075, TDist(2.5)), 100_000)
quantile(simt, [.02, .25, .5, .75, .98])

5-element Vector{Float64}:
 -0.2628354155012672
 -0.019101057077905423
  0.03930843980215
  0.09881095316279048
  0.3360791286156967